In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

In [3]:
df = pd.read_csv("accidents.csv")
df.drop('Unnamed: 0', 1, inplace=True)

In [8]:
df.shape

(36016, 27)

In [9]:
df.head()

,Accident_Time,Accident_Date_G,Rno,Road_Type,Region,Road_Status,Weather_Status,Dead_Number,Injuries_Number,Damage_Road_Type,...,Vehicle,Animal,Vehicles_Number,Paints,Eyes,Road_Close_Days,Road_Close_Hours,GEOMETRIC_RoadTYPE,long,lat
0,5:00 AM,16/04/2019,425,سريع مع خدمة,Qassim,Good,good,0.0,1.0,NaN,...,NaN,NaN,1.0,1.0,1.0,NaN,NaN,Straight link,43.999093,26.426746
1,12:15 PM,16/04/2019,414,مزدوج,Qassim,Good,good,0.0,1.0,NaN,...,NaN,NaN,2.0,1.0,1.0,NaN,NaN,Straight link,43.786877,26.057596
2,8:30 AM,15/04/2019,419,مزدوج,Qassim,Good,good,0.0,0.0,NaN,...,NaN,NaN,2.0,1.0,1.0,NaN,NaN,Straight link,43.772701,26.345146
3,8:30 PM,15/04/2019,60,سريع مع خدمة,Qassim,Good,good,0.0,0.0,NaN,...,NaN,NaN,2.0,1.0,1.0,NaN,NaN,Straight link,43.937996,26.205702
4,12:10 AM,14/04/2019,393,NaN,Hail,Good,good,NaN,NaN,NaN,...,NaN,NaN,2.0,1.0,1.0,NaN,NaN,Straight link,41.702023,27.555157


In [10]:
def add_datepart(df, field_name:str, drop:bool=True, time:bool=False):
    "Helper function that adds columns relevant to a date in the column `field_name` of `df`."         
    field = df[field_name]                                                                             
    prefix = re.sub('[Dd]ate$', '', field_name)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start',
            'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start'] 
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[prefix + n] = getattr(field.dt, n.lower())
    df[prefix + 'Elapsed'] = field.astype(np.int64) // 10 ** 9
    if drop: df.drop(field_name, axis=1, inplace=True) 
    return df

In [162]:
df['acc_datetime'] = pd.to_datetime(df.Accident_Date_G+' '+df.Accident_Time, format='%d/%m/%Y %I:%M %p')
df['acc_date'] = pd.to_datetime(df.Accident_Date_G, format='%d/%m/%Y')
df = add_datepart(df, field_name='acc_datetime', drop=False, time=True)
# df['acc_time']`

In [163]:
df.loc[:, ['Accident_Date_G', 'Accident_Time', 'acc_datetime', 'acc_date', 'acc_datetimeMonth']].sample(10)

,Accident_Date_G,Accident_Time,acc_datetime,acc_date,acc_datetimeMonth
6428,19/10/2018,6:05 PM,2018-10-19 18:05:00,2018-10-19,10
4294,10/12/2018,9:00 PM,2018-12-10 21:00:00,2018-12-10,12
22102,14/10/2017,9:00 AM,2017-10-14 09:00:00,2017-10-14,10
21075,04/11/2017,3:30 PM,2017-11-04 15:30:00,2017-11-04,11
32944,04/03/2017,12:00 PM,2017-03-04 12:00:00,2017-03-04,3
19099,16/12/2017,8:20 AM,2017-12-16 08:20:00,2017-12-16,12
13028,08/05/2018,10:40 AM,2018-05-08 10:40:00,2018-05-08,5
16342,24/02/2018,7:30 PM,2018-02-24 19:30:00,2018-02-24,2
9915,20/07/2018,4:30 PM,2018-07-20 16:30:00,2018-07-20,7
20780,11/11/2017,7:30 PM,2017-11-11 19:30:00,2017-11-11,11


In [143]:
df.columns


Index(['Accident_Time', 'Accident_Date_G', 'Rno', 'Road_Type', 'Region',
       'Road_Status', 'Weather_Status', 'Dead_Number', 'Injuries_Number',
       'Damage_Road_Type', 'Accident_Type', 'Car', 'Small_Truck', 'Big_Truck',
       'Other_Car', 'Driver', 'Way', 'Vehicle', 'Animal', 'Vehicles_Number',
       'Paints', 'Eyes', 'Road_Close_Days', 'Road_Close_Hours',
       'GEOMETRIC_RoadTYPE', 'long', 'lat', 'acc_datetime', 'acc_date',
       'acc_datetimeYear', 'acc_datetimeMonth', 'acc_datetimeWeek',
       'acc_datetimeDay', 'acc_datetimeDayofweek', 'acc_datetimeDayofyear',
       'acc_datetimeIs_month_end', 'acc_datetimeIs_month_start',
       'acc_datetimeIs_quarter_end', 'acc_datetimeIs_quarter_start',
       'acc_datetimeIs_year_end', 'acc_datetimeIs_year_start',
       'acc_datetimeHour', 'acc_datetimeMinute', 'acc_datetimeSecond',
       'acc_datetimeElapsed'],
      dtype='object')

In [194]:
# df['Region'].value_counts(normalize=True)
# df.Car
# df.groupby('Accident_Type')[['Dead_Number', 'Injuries_Number']].sum().\
#                         sort_values('Dead_Number', ascending=False)
pd.pivot_table(df.groupby(['Accident_Type', 'Road_Type']).size().to_frame().reset_index(), 
               values=0, 
               columns='Road_Type', 
               index='Accident_Type', 
               aggfunc='sum', 
               fill_value=0,
               margins=True)

Road_Type,سريع,سريع اتجاهين,سريع مع خدمة,عقبة,مزدوج,مفرد,All
Accident_Type,,,,,,,
Explosion of a car tire,508,0,11,2,274,157,952
burning or ignition,301,2,8,11,209,131,662
collateral crush,127,0,1,0,86,29,243
coup,2742,25,29,38,3198,1974,8006
coup and crash,293,0,9,2,112,53,469
crash,6703,10,33,127,5309,2922,15104
crush from behind,1008,0,62,0,593,134,1797
crush with a stationary body on the road,145,0,0,6,141,57,349
deflection,1498,16,51,31,898,449,2943


In [191]:
pd.pivot_table(df.groupby(['Accident_Type', 'Road_Type'])['Dead_Number'].sum().to_frame().reset_index(),
                values='Dead_Number', 
               columns='Road_Type', 
               index='Accident_Type', 
               aggfunc='sum', 
               fill_value=0,
               margins=True)

Road_Type,سريع,سريع اتجاهين,سريع مع خدمة,عقبة,مزدوج,مفرد,All
Accident_Type,,,,,,,
Explosion of a car tire,33,0,0,0,29,17,79.0
burning or ignition,14,0,0,0,8,11,33.0
collateral crush,0,0,0,0,3,6,9.0
coup,278,4,2,21,434,346,1085.0
coup and crash,56,0,0,0,25,16,97.0
crash,478,1,0,9,671,994,2153.0
crush from behind,62,0,2,0,52,15,131.0
crush with a stationary body on the road,4,0,0,7,9,3,23.0
deflection,44,1,1,4,55,41,146.0


In [140]:
df['GEOMETRIC_RoadTYPE'].value_counts()

Straight link         17145
Horizontal Curve       1165
Vertical Curve          168
u-tern                  124
3-Leg intersection       61
High Slope Road          51
4-Leg intersection       50
Interchange              25
Tunnel                   14
Name: GEOMETRIC_RoadTYPE, dtype: int64

In [202]:
monthly_df = df.resample('M', on='acc_datetime')[['Dead_Number', 'Injuries_Number']].\
                sum().reset_index()
monthly_df['month'] = getattr(monthly_df['acc_datetime'].dt, 'month')
# monthly_df.groupby()

,acc_datetime,Dead_Number,Injuries_Number,month
0,2017-01-31,211.0,1318.0,1
1,2017-02-28,167.0,1231.0,2
2,2017-03-31,162.0,1228.0,3
3,2017-04-30,207.0,1182.0,4
4,2017-05-31,217.0,1318.0,5
5,2017-06-30,264.0,1488.0,6
6,2017-07-31,271.0,1176.0,7
7,2017-08-31,179.0,1140.0,8
8,2017-09-30,191.0,1155.0,9
9,2017-10-31,168.0,1130.0,10


In [2]:
monthly_df.plot.bar(x='acc_datetime', y=['Dead_Number', 'Injuries_Number'], figsize=(10,5))

NameError: name 'monthly_df' is not defined

In [200]:
monthly_df['acc_datetime']

0    2017-01-31
1    2017-02-28
2    2017-03-31
3    2017-04-30
4    2017-05-31
5    2017-06-30
6    2017-07-31
7    2017-08-31
8    2017-09-30
9    2017-10-31
10   2017-11-30
11   2017-12-31
12   2018-01-31
13   2018-02-28
14   2018-03-31
15   2018-04-30
16   2018-05-31
17   2018-06-30
18   2018-07-31
19   2018-08-31
20   2018-09-30
21   2018-10-31
22   2018-11-30
23   2018-12-31
24   2019-01-31
25   2019-02-28
26   2019-03-31
27   2019-04-30
Name: acc_datetime, dtype: datetime64[ns]

In [72]:
df.Accident_Type.value_count()

AttributeError: 'Series' object has no attribute 'value_count'

In [158]:
df.columns

Index(['Accident_Time', 'Accident_Date_G', 'Rno', 'Road_Type', 'Region',
       'Road_Status', 'Weather_Status', 'Dead_Number', 'Injuries_Number',
       'Damage_Road_Type', 'Accident_Type', 'Car', 'Small_Truck', 'Big_Truck',
       'Other_Car', 'Driver', 'Way', 'Vehicle', 'Animal', 'Vehicles_Number',
       'Paints', 'Eyes', 'Road_Close_Days', 'Road_Close_Hours',
       'GEOMETRIC_RoadTYPE', 'long', 'lat', 'acc_datetime', 'acc_date',
       'acc_datetimeYear', 'acc_datetimeMonth', 'acc_datetimeWeek',
       'acc_datetimeDay', 'acc_datetimeDayofweek', 'acc_datetimeDayofyear',
       'acc_datetimeIs_month_end', 'acc_datetimeIs_month_start',
       'acc_datetimeIs_quarter_end', 'acc_datetimeIs_quarter_start',
       'acc_datetimeIs_year_end', 'acc_datetimeIs_year_start',
       'acc_datetimeHour', 'acc_datetimeMinute', 'acc_datetimeSecond',
       'acc_datetimeElapsed'],
      dtype='object')

In [175]:
df.groupby('acc_datetimeMonth')[['Dead_Number', 'Injuries_Number']].agg(['sum', 'mean', 'count'])


Dead_Number                 Injuries_Number                
                          sum      mean count             sum      mean count
acc_datetimeMonth                                                            
1                       496.0  0.205809  2410          3397.0  1.127448  3013
2                       417.0  0.168145  2480          2906.0  0.970608  2994
3                       398.0  0.141285  2817          3114.0  0.946505  3290
4                       370.0  0.155332  2382          2447.0  0.896337  2730
5                       394.0  0.187798  2098          2476.0  1.004462  2465
6                       453.0  0.214184  2115          2693.0  1.115114  2415
7                       449.0  0.229082  1960          2197.0  0.976879  2249
8                       340.0  0.193843  1754          2211.0  1.071740  2063
9                       333.0  0.209040  1593          1986.0  1.010173  1966
10                      316.0  0.179852  1757          2094.0  0.991008  2113
11                      311.0  0.181130  1717          2017.0  0.962768  2095
12                      293.0  0.178224  1644          2024.0  1.016575  1991